# **Install segmentation_models_pytorch**

In [1]:
!pip install segmentation_models_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.7 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=6bcf78b44d4f37078280b8903423a98517614f649f39bb83f8a79b5db79bb3ef
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=625bef912f12cc3c3d9066befe09e3c12a6fc8777146a7db5e2e850dda63c01b
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built efficient

# **Import Libraries**

In [2]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import cv2
import glob
import os
import re
import shutil
import timm
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.cuda import amp
import albumentations as A
from albumentations.pytorch import ToTensorV2
import transformers
from sklearn.model_selection import StratifiedKFold, KFold, StratifiedGroupKFold, GroupKFold
import multiprocessing as mp
import segmentation_models_pytorch as smp
import copy
from collections import defaultdict
import gc
from tqdm import tqdm
import tifffile
from colorama import Fore, Back, Style
c_  = Fore.GREEN
sr_ = Style.RESET_ALL

# **Config**

In [3]:
class CFG:
    seed = 0
    batch_size = 16
    head = "UNet"
    backbone = "efficientnet-b3" #['efficientnet-b0', efficientnet-b1', ... , efficientnet-b7'] and many other backbone architectures
    img_size = [256, 256] # Or [512, 512]
    lr = 1e-3
    scheduler = 'CosineAnnealingLR' #['CosineAnnealingLR', 'ReduceLROnPlateau', 'ExponentialLR']
    epochs = 20
    warmup_epochs = 2
    n_folds = 5
    folds_to_run = [0, 1, 2, 3, 4]
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    base_path = '/kaggle/input/hubmap-organ-segmentation'
    crops_path = '/kaggle/input/hubmap-2022-256x256' # Or '/kaggle/input/hubmap-2022-512x512'
    num_workers = mp.cpu_count()
    num_classes = 1
    n_accumulate = max(1, 16//batch_size)
    loss = 'Dice'
    optimizer = 'AdamW'
    weight_decay = 1e-6

# **Helper Functions**

In [4]:
# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction


# ref.: https://www.kaggle.com/stainsby/fast-tested-rle
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [5]:
def read_tiff(path, scale=None, verbose=0): #Modified from https://www.kaggle.com/code/abhinand05/hubmap-extensive-eda-what-are-we-hacking
    image = tifffile.imread(path)
    if len(image.shape) == 5:
        image = image.squeeze().transpose(1, 2, 0)
    
    if verbose:
        print(f"[{path}] Image shape: {image.shape}")
    
    if scale:
        new_size = (image.shape[1] // scale, image.shape[0] // scale)
        image = cv2.resize(image, new_size)
        
        if verbose:
            print(f"[{path}] Resized Image shape: {image.shape}")
        
#     mx = np.max(image)
#     image = image.astype(np.float32)
#     if mx:
#         image /= mx # scale image to [0, 1]
    return image

def read_img(path, verbose=0):
    image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    
    if verbose:
        print(f"[{path}] Image shape: {image.shape}")
        
#     mx = np.max(image)
#     image = image.astype(np.float32)
#     if mx:
#         image /= mx
        
    return image

In [6]:
def prepare_loaders(fold):
    train_df = df.query("fold!=@fold").reset_index(drop=True)
    valid_df = df.query("fold==@fold").reset_index(drop=True)

    train_dataset = HuBMAP_Dataset(train_df, transforms=data_transforms['train'])
    valid_dataset = HuBMAP_Dataset(valid_df, transforms=data_transforms['valid'])

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=CFG.batch_size,
                              num_workers=CFG.num_workers, shuffle=True, pin_memory=True, drop_last=False)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=CFG.batch_size,
                              num_workers=CFG.num_workers, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

# **Set Seed For Reproducibility**

In [7]:
def set_seed(seed = 42): #From https://www.kaggle.com/code/awsaf49/uwmgi-unet-train-pytorch/
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    print('> SEEDING DONE')
    
set_seed(CFG.seed)

> SEEDING DONE


# **Grab Metadata**

In [8]:
comp_train_df = pd.read_csv(os.path.join(CFG.base_path, 'train.csv'))
comp_train_df.head()

,id,organ,data_source,img_height,img_width,pixel_size,tissue_thickness,rle,age,sex
0,10044,prostate,HPA,3000,3000,0.4,4,1459676 77 1462675 82 1465674 87 1468673 92 14...,37.0,Male
1,10274,prostate,HPA,3000,3000,0.4,4,715707 2 718705 8 721703 11 724701 18 727692 3...,76.0,Male
2,10392,spleen,HPA,3000,3000,0.4,4,1228631 20 1231629 24 1234624 40 1237623 47 12...,82.0,Male
3,10488,lung,HPA,3000,3000,0.4,4,3446519 15 3449517 17 3452514 20 3455510 24 34...,78.0,Male
4,10610,spleen,HPA,3000,3000,0.4,4,478925 68 481909 87 484893 105 487863 154 4908...,21.0,Female


In [9]:
images_list = list(glob.glob(f'{CFG.crops_path}/train/*.png'))
print(images_list[:5])

['/kaggle/input/hubmap-2022-256x256/train/18445_0003.png', '/kaggle/input/hubmap-2022-256x256/train/28052_0001.png', '/kaggle/input/hubmap-2022-256x256/train/6794_0003.png', '/kaggle/input/hubmap-2022-256x256/train/1184_0002.png', '/kaggle/input/hubmap-2022-256x256/train/29223_0004.png']


In [10]:
df = pd.DataFrame({
    "id": [x.split('/')[-1].split('_')[0] for x in images_list],
    "image_paths": images_list,
    "mask_paths": [x.replace('train', 'masks') for x in images_list],
    "organ": [comp_train_df[comp_train_df['id'] == int(x.split('/')[-1].split('_')[0])]['organ'].item() for x in images_list]
    #You can grab any metadata from the original train.csv for each cropped image by modifying ['organ'] to the column you want above
})
df.head()

,id,image_paths,mask_paths,organ
0,18445,/kaggle/input/hubmap-2022-256x256/train/18445_...,/kaggle/input/hubmap-2022-256x256/masks/18445_...,largeintestine
1,28052,/kaggle/input/hubmap-2022-256x256/train/28052_...,/kaggle/input/hubmap-2022-256x256/masks/28052_...,lung
2,6794,/kaggle/input/hubmap-2022-256x256/train/6794_0...,/kaggle/input/hubmap-2022-256x256/masks/6794_0...,prostate
3,1184,/kaggle/input/hubmap-2022-256x256/train/1184_0...,/kaggle/input/hubmap-2022-256x256/masks/1184_0...,prostate
4,29223,/kaggle/input/hubmap-2022-256x256/train/29223_...,/kaggle/input/hubmap-2022-256x256/masks/29223_...,prostate


# **Data Processing**

In [11]:
kf = GroupKFold(n_splits=CFG.n_folds)
for fold, (train_idx, val_idx) in enumerate(kf.split(df, groups=df['id'])):
    df.loc[val_idx, 'fold'] = fold
df.head()

,id,image_paths,mask_paths,organ,fold
0,18445,/kaggle/input/hubmap-2022-256x256/train/18445_...,/kaggle/input/hubmap-2022-256x256/masks/18445_...,largeintestine,2.0
1,28052,/kaggle/input/hubmap-2022-256x256/train/28052_...,/kaggle/input/hubmap-2022-256x256/masks/28052_...,lung,4.0
2,6794,/kaggle/input/hubmap-2022-256x256/train/6794_0...,/kaggle/input/hubmap-2022-256x256/masks/6794_0...,prostate,2.0
3,1184,/kaggle/input/hubmap-2022-256x256/train/1184_0...,/kaggle/input/hubmap-2022-256x256/masks/1184_0...,prostate,3.0
4,29223,/kaggle/input/hubmap-2022-256x256/train/29223_...,/kaggle/input/hubmap-2022-256x256/masks/29223_...,prostate,3.0


# **Dataset**

In [12]:
class HuBMAP_Dataset(torch.utils.data.Dataset):
    def __init__(self, df, labeled=True, transforms=None):
        self.df = df
        self.labeled = labeled
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = self.df.loc[index, 'image_paths']
        img = read_img(img_path)
        
        if self.labeled:
            mask_path = self.df.loc[index, 'mask_paths']
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            
            if self.transforms:
                data = self.transforms(image=img, mask=mask)
                img  = data['image']
                mask  = data['mask']
            
            mask = np.expand_dims(mask, axis=0)
            img = np.transpose(img, (2, 0, 1))
#             mask = np.transpose(mask, (2, 0, 1))
            
            return torch.tensor(img), torch.tensor(mask)
        
        else:
            if self.transforms:
                data = self.transforms(image=img)
                img  = data['image']
                
            img = np.transpose(img, (2, 0, 1))
            
            return torch.tensor(img)

# **Augmentations**

In [13]:
data_transforms = {
    "train": A.Compose([
        A.Resize(*CFG.img_size, interpolation=cv2.INTER_NEAREST),
        A.HorizontalFlip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.05, rotate_limit=10, p=0.5),
        A.RandomRotate90(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=15, p=0.5, border_mode=cv2.BORDER_REFLECT),
        A.OneOf([
            A.HueSaturationValue(10,15,10),
            A.CLAHE(clip_limit=2),
            A.RandomBrightnessContrast(),            
        ], p=0.4),
        A.Normalize(),
    ]),
    
    "valid": A.Compose([
        A.Resize(*CFG.img_size, interpolation=cv2.INTER_NEAREST),
        A.Normalize(),
        ], p=1.0),
}

# **Losses and Metrics**

In [14]:
JaccardLoss = smp.losses.JaccardLoss(mode='binary')
DiceLoss    = smp.losses.DiceLoss(mode='binary')
BCELoss     = smp.losses.SoftBCEWithLogitsLoss()
LovaszLoss  = smp.losses.LovaszLoss(mode='binary', per_image=False)
TverskyLoss = smp.losses.TverskyLoss(mode='binary', log_loss=False)

def dice_coef(y_true, y_pred, thr=0.5, dim=(2,3), epsilon=0.001):
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred>thr).to(torch.float32)
    inter = (y_true*y_pred).sum(dim=dim)
    den = y_true.sum(dim=dim) + y_pred.sum(dim=dim)
    dice = ((2*inter+epsilon)/(den+epsilon)).mean(dim=(1,0))
    return dice

def iou_coef(y_true, y_pred, thr=0.5, dim=(2,3), epsilon=0.001):
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred>thr).to(torch.float32)
    inter = (y_true*y_pred).sum(dim=dim)
    union = (y_true + y_pred - y_true*y_pred).sum(dim=dim)
    iou = ((inter+epsilon)/(union+epsilon)).mean(dim=(1,0))
    return iou

losses = {
    "Dice": DiceLoss,
    "Jaccard": JaccardLoss,
    "BCE": BCELoss,
    "Lovasz": LovaszLoss,
    "Tversky": TverskyLoss,
}

# **Schedulers**

In [15]:
def get_scheduler(optimizer):
    
    if len(df[df['fold'] == CFG.folds_to_run[0]]) % CFG.batch_size != 0:
        num_steps = len(df[df['fold'] != CFG.folds_to_run[0]]) // CFG.batch_size + 1
    
    else:
        len(df[df['fold'] != CFG.folds_to_run[0]]) // CFG.batch_size
    
    if CFG.scheduler == 'CosineAnnealingLR':
        scheduler = transformers.get_cosine_schedule_with_warmup(optimizer, CFG.warmup_epochs * num_steps, CFG.epochs * num_steps)
        
    elif CFG.scheduler == 'ReduceLROnPlateau':
        scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=7, threshold=0.0001, min_lr=1e-6)
    elif CFG.scheduer == 'ExponentialLR':
        scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.85)
        
    return scheduler

# **Optimizers**

In [16]:
def get_optimizer(optimizer_name=CFG.optimizer):
    if CFG.optimizer == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=CFG.lr)
    
    elif CFG.optimizer == 'AdamW':
        optimizer = optim.AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
        
    return optimizer

# **Models**

In [17]:
def build_model():
    model = smp.Unet(
        encoder_name=CFG.backbone,      
        encoder_weights="imagenet",     
        in_channels=3,                  
        classes=CFG.num_classes,
        activation=None,
    )
    model.to(CFG.device)
    return model

def load_model(path):
    model = build_model()
    model.load_state_dict(torch.load(path))
    model.eval()
    return model

# **Training Functions**

Modified from https://www.kaggle.com/code/awsaf49/uwmgi-unet-train-pytorch/

In [18]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    scaler = amp.GradScaler()
    
    dataset_size = 0
    running_loss = 0.0
    criterion = losses[CFG.loss]
    
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Train ')
    for step, (images, masks) in pbar:         
        images = images.to(device, dtype=torch.float)
        masks  = masks.to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        
        with amp.autocast(enabled=True):
            y_pred = model(images)
            loss   = criterion(y_pred, masks)
            
        scaler.scale(loss).backward()
    
        if (step + 1) % CFG.n_accumulate == 0:
            scaler.step(optimizer)
            scaler.update()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(train_loss=f'{epoch_loss:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_mem=f'{mem:0.2f} GB')
    torch.cuda.empty_cache()
    gc.collect()
    
    return epoch_loss

In [19]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    criterion = losses[CFG.loss]
    
    val_scores = []
    
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Valid ')
    for step, (images, masks) in pbar:        
        images  = images.to(device, dtype=torch.float)
        masks   = masks.to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        
        y_pred  = model(images)
        loss    = criterion(y_pred, masks)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        y_pred = nn.Sigmoid()(y_pred)
        val_dice = dice_coef(masks, y_pred).cpu().detach().numpy()
        val_jaccard = iou_coef(masks, y_pred).cpu().detach().numpy()
        val_scores.append([val_dice, val_jaccard])
        
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(valid_loss=f'{epoch_loss:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_memory=f'{mem:0.2f} GB')
    val_scores  = np.mean(val_scores, axis=0)
    torch.cuda.empty_cache()
    gc.collect()
    
    return epoch_loss, val_scores

In [20]:
def run_training(model, optimizer, scheduler, device, num_epochs):
    # To automatically log gradients
    
    if torch.cuda.is_available():
        print("cuda: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_dice      = -np.inf
    best_epoch     = -1
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        print(f'Epoch {epoch}/{num_epochs}', end='')
        train_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CFG.device, epoch=epoch)
        
        val_loss, val_scores = valid_one_epoch(model, valid_loader, 
                                                 device=CFG.device, 
                                                 epoch=epoch)
        val_dice, val_jaccard = val_scores
    
        history['Train Loss'].append(train_loss)
        history['Valid Loss'].append(val_loss)
        history['Valid Dice'].append(val_dice)
        history['Valid Jaccard'].append(val_jaccard)
        
        print(f'Valid Dice: {val_dice:0.4f} | Valid Jaccard: {val_jaccard:0.4f}')
        
        # deep copy the model
        if val_dice >= best_dice:
            print(f"{c_}Valid Score Improved ({best_dice:0.4f} ---> {val_dice:0.4f})")
            best_dice    = val_dice
            best_jaccard = val_jaccard
            best_epoch   = epoch
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f"best_epoch-{fold:02d}.bin"
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print(f"Model Saved{sr_}")
            
        last_model_wts = copy.deepcopy(model.state_dict())
        PATH = f"last_epoch-{fold:02d}.bin"
        torch.save(model.state_dict(), PATH)
            
        print(); print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Score: {:.4f}".format(best_dice))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

# **Training**

In [ ]:
for fold in CFG.folds_to_run:
    print(f'#'*15)
    print(f'### Fold: {fold}')
    print(f'#'*15)
    train_loader, valid_loader = prepare_loaders(fold=fold)
    model = build_model()
    optimizer = get_optimizer()
    scheduler = get_scheduler(optimizer)
    model, history = run_training(model, optimizer, scheduler,
                                  device=CFG.device,
                                  num_epochs=CFG.epochs)

###############
### Fold: 0
###############


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth


  0%|          | 0.00/47.1M [00:00<?, ?B/s]

cuda: Tesla T4

Epoch 1/20

Valid : 100%|██████████| 35/35 [00:07<00:00,  4.80it/s, gpu_memory=1.22 GB, lr=0.00050, valid_loss=0.4105]


...

Valid : 100%|██████████| 35/35 [00:05<00:00,  6.69it/s, gpu_memory=1.23 GB, lr=0.00018, valid_loss=0.1760]


Valid Dice: 0.7668 | Valid Jaccard: 0.7216


Epoch 16/20

Valid : 100%|██████████| 35/35 [00:05<00:00,  6.75it/s, gpu_memory=1.23 GB, lr=0.00012, valid_loss=0.1786]


Valid Dice: 0.7739 | Valid Jaccard: 0.7287
Valid Score Improved (0.7711 ---> 0.7739)
Model Saved


Epoch 17/20

Train :   4%|▍         | 6/140 [00:02<00:44,  2.99it/s, gpu_mem=3.24 GB, lr=0.00011, train_loss=0.1766]